In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql import Row

# constants for connection
JDBC_POSTGRES_VERSION = '42.7.3'
POSTGRES_HOST_NAME = 'localhost'
POSTGRES_PORT=5432
POSTGRES_DBNAME='postgres_test'
POSTGRES_USERNAME = 'airflow'
POSTGRES_PASSWORD = 'airflow'

In [2]:
# the Spark session should be instantiated as follows
spark = SparkSession \
    .builder \
    .appName("Python Spark PostgreSQL basic example") \
    .master("local") \
    .config("spark.jars", f"{JDBC_POSTGRES_VERSION}.jar") \
    .getOrCreate()

In [3]:
# connect to our table and show data
df = spark.read.format("jdbc"). \
options(url=f'jdbc:postgresql://{POSTGRES_HOST_NAME}:{POSTGRES_PORT}/{POSTGRES_DBNAME}', # jdbc:postgresql://<host>:<port>/<database>
        dbtable='pet',
        user=POSTGRES_USERNAME,
        password=POSTGRES_PASSWORD,
        driver='org.postgresql.Driver').load()
df.show()

+------+------+--------+----------+-----+
|pet_id|  name|pet_type|birth_date|owner|
+------+------+--------+----------+-----+
|     1|   Max|     Dog|2018-07-05| Jane|
|     2| Susie|     Cat|2019-05-01| Phil|
|     3|Lester| Hamster|2020-06-23| Lily|
|     4|Quincy|  Parrot|2013-08-11| Anne|
+------+------+--------+----------+-----+



In [4]:
# create table in PySpark
student_df = spark.createDataFrame([
	Row(id=1,name='vijay',marks=67),
	Row(id=2,name='Ajay',marks=88),
	Row(id=3,name='jay',marks=79),
	Row(id=4,name='vinay',marks=67),
])
student_df.show()

+---+-----+-----+
| id| name|marks|
+---+-----+-----+
|  1|vijay|   67|
|  2| Ajay|   88|
|  3|  jay|   79|
|  4|vinay|   67|
+---+-----+-----+



In [6]:
# test writting to database
student_df.select("id","name","marks").write.format("jdbc")\
    .option("url", f"jdbc:postgresql://{POSTGRES_HOST_NAME}:{POSTGRES_PORT}/{POSTGRES_DBNAME}") \
    .option("driver", "org.postgresql.Driver").option("dbtable", "students") \
    .option("user", POSTGRES_USERNAME).option("password", POSTGRES_PASSWORD).save()